In [1]:
import pandas as pd                    
import xml.etree.ElementTree as ET
import unicodedata

N_CVs = 53
linhas = [] # uma linha para cada artigo lido
base_path = '../xml/'

researcher_df = pd.DataFrame(columns=['name', 'research_area', 'program'])
base_research_field = pd.DataFrame(columns=['name', 'research_area'])
institution = pd.DataFrame(columns=['institution_name'])
production = pd.DataFrame(columns=['event_name', 'event_qualis', 'event_type'])
time = pd.DataFrame(columns=['year'])
current_researcher = ""

In [2]:
def normalize_str(source, encoding):
    if encoding  == 'ISO-8859-1':
        target = str.encode(source, 'ISO-8859-1')
        target = target.decode('UTF-8')
    else:
        target = str(source)
        
    target = target.strip()
    
    target = ''.join(c for c in unicodedata.normalize('NFD', target)
                  if unicodedata.category(c) != 'Mn')
    
    return target.upper()

In [3]:
def parse_researcher_data(xmlelement, field_df, debug=False):
    name = normalize_str(xmlelement.get('NOME-COMPLETO'), 'ISO-8859-1')
    program = "PPGC" # Can change later to add a properly method to fetch
    
    field = field_df[field_df['name'] == name].iloc[0]['research_area']
    
    if debug:
        print("RESEARCHER [" + name + "]\t[" + program + "]\t[" + field + "]")
    return [name, field, program]

In [4]:
def parse_production(linhas, elem):
    trabalhos = elem.findall('./TRABALHOS-EM-EVENTOS/TRABALHO-EM-EVENTOS')
    for trabalho in trabalhos:
        dados_basicos = trabalho.find('DADOS-BASICOS-DO-TRABALHO').attrib
        detalhes = trabalho.find('DETALHAMENTO-DO-TRABALHO').attrib
        autores = trabalho.findall('AUTORES')
        if dados_basicos.get('NATUREZA') == 'COMPLETO':
            dados = dict(dados_basicos)
            dados.update(detalhes)
            lst = []
            lst_autores= {}
            for autor in autores:                        
                lst.append(autor.get('NOME-COMPLETO-DO-AUTOR'))                      
            lst_autores['AUTORES'] = lst
            dados.update(lst_autores)                    
            linhas.append(dados)

In [5]:
periodics = pd.read_csv(base_path + "QualisPeriodicos.csv", sep=';', encoding='ISO-8859-1')
conferences = pd.read_csv(base_path + "QualisConferencias.csv", sep=';', encoding='ISO-8859-1')
for idx, val in periodics.iterrows():
    periodics.iloc[idx,1] = normalize_str(periodics.iloc[idx,1], 'UTF-8')
    periodics.iloc[idx,2] = normalize_str(periodics.iloc[idx,2], 'UTF-8')

for idx, val in conferences.iterrows():
    conferences.iloc[idx,0] = normalize_str(conferences.iloc[idx,0], 'UTF-8')
    conferences.iloc[idx,1] = normalize_str(conferences.iloc[idx,1], 'UTF-8')
    conferences.iloc[idx,3] = normalize_str(conferences.iloc[idx,3], 'UTF-8')

In [6]:
with open(base_path + "Linhas_de_Pesquisa.csv", "r") as file:
    next(file)
    iter = 0
    for line in file:
        arr = line.split(';')
        base_research_field.loc[iter] = [normalize_str(arr[2], 'UTF-8'), arr[1]]
        iter += 1

In [7]:
production_df = None
for i in range(1,N_CVs+1):
    arquivo = base_path + 'curriculo(' + str(i) + ').xml'
    print("Reading file: ", arquivo)
    doc_tree_root = ET.parse(arquivo).getroot()
    for elem in doc_tree_root:                
        if elem.tag == 'PRODUCAO-BIBLIOGRAFICA':
            parse_production(linhas, elem)
        # Geração de tabela 
        if elem.tag == 'DADOS-GERAIS':
            researcher_df.loc[i-1] = parse_researcher_data(elem, base_research_field)
            current_researcher = researcher_df.iloc[i-1, 0]
            
    print("The file belongs to " + current_researcher)        
    
    prodPPGC = pd.DataFrame(linhas)
    # remove colunas não utilizadas
    prodPPGC.drop(['ANO-DE-REALIZACAO', 'AUTORES', 'CIDADE-DA-EDITORA', 'CIDADE-DO-EVENTO', 'CLASSIFICACAO-DO-EVENTO',
              'DOI', 'FASCICULO', 'FLAG-DIVULGACAO-CIENTIFICA', 'FLAG-RELEVANCIA', 'HOME-PAGE-DO-TRABALHO', 'IDIOMA', 'ISBN',
              'MEIO-DE-DIVULGACAO', 'NATUREZA', 'NOME-DA-EDITORA', 'PAGINA-FINAL', 'PAGINA-INICIAL', 'PAIS-DO-EVENTO',
              'SERIE', 'TITULO-DOS-ANAIS-OU-PROCEEDINGS', 'VOLUME', 'TITULO-DO-TRABALHO-INGLES'], axis=1, inplace=True)
    
    prodPPGC['EVENT-ABREVIATION'] = ""
    prodPPGC['EVENT-ISSN'] = ""
    prodPPGC['QUALIS'] = ""
    prodPPGC['PERSON'] = current_researcher
    
    for idx, val in prodPPGC.iterrows():
        prodPPGC.iloc[idx,1] = normalize_str(prodPPGC.iloc[idx,1], 'ISO-8859-1')
        prodPPGC.iloc[idx,2] = normalize_str(prodPPGC.iloc[idx,2], 'ISO-8859-1')
        
    for idx, cols in prodPPGC.iterrows():
        res = conferences[conferences['Nome'] == prodPPGC.iloc[idx,1]]
        if res.empty == False:
            prodPPGC.iloc[idx, 3] = res.iloc[0, 0]
            prodPPGC.iloc[idx, 5] = res.iloc[0, 3]
        else:
            res = periodics[periodics['Nome'] == prodPPGC.iloc[idx,1]]
            if res.empty == False:
                prodPPGC.iloc[idx, 4] = res.iloc[0, 0]
                prodPPGC.iloc[idx, 5] = res.iloc[0, 2]
        if res.empty:
            prodPPGC.iloc[idx, 5] = "NAN"
    
    if production_df is None:
        production_df = prodPPGC.copy()
    else:
        production_df = production_df.append(prodPPGC)

Reading file:  ../xml/curriculo(1).xml
The file belongs to RENATO PEREZ RIBAS
Reading file:  ../xml/curriculo(2).xml
The file belongs to VALTER ROESLER
Reading file:  ../xml/curriculo(3).xml
The file belongs to LUIS DA CUNHA LAMB
Reading file:  ../xml/curriculo(4).xml
The file belongs to INGRID OLIVEIRA DE NUNES
Reading file:  ../xml/curriculo(5).xml
The file belongs to ERIKA FERNANDES COTA
Reading file:  ../xml/curriculo(6).xml
The file belongs to CLAUDIO ROSITO JUNG
Reading file:  ../xml/curriculo(7).xml
The file belongs to KARIN BECKER
Reading file:  ../xml/curriculo(8).xml
The file belongs to DALTRO JOSE NUNES
Reading file:  ../xml/curriculo(9).xml
The file belongs to JACOB SCHARCANSKI
Reading file:  ../xml/curriculo(10).xml
The file belongs to NICOLAS MAILLARD
Reading file:  ../xml/curriculo(11).xml
The file belongs to MARCELO SOARES PIMENTA
Reading file:  ../xml/curriculo(12).xml
The file belongs to EDSON PRESTES E SILVA JUNIOR
Reading file:  ../xml/curriculo(13).xml
The file bel

In [11]:
production_df[production_df['TITULO-DO-TRABALHO'] == 'A TOOL FOR ANALYSIS OF UNIVERSAL LOGIC GATES FUNCTIONALITY']

,ANO-DO-TRABALHO,NOME-DO-EVENTO,TITULO-DO-TRABALHO,EVENT-ABREVIATION,EVENT-ISSN,QUALIS,PERSON
3314,1999,A TOOL FOR ANALYSIS OF UNIVERSAL LOGIC GATES F...,A TOOL FOR ANALYSIS OF UNIVERSAL LOGIC GATES F...,,,NAN,FERNANDA GUSMAO DE LIMA KASTENSMIDT
3314,1999,A TOOL FOR ANALYSIS OF UNIVERSAL LOGIC GATES F...,A TOOL FOR ANALYSIS OF UNIVERSAL LOGIC GATES F...,,,NAN,ALBERTO EGON SCHAEFFER FILHO
3314,1999,A TOOL FOR ANALYSIS OF UNIVERSAL LOGIC GATES F...,A TOOL FOR ANALYSIS OF UNIVERSAL LOGIC GATES F...,,,NAN,MARA ABEL
3314,1999,A TOOL FOR ANALYSIS OF UNIVERSAL LOGIC GATES F...,A TOOL FOR ANALYSIS OF UNIVERSAL LOGIC GATES F...,,,NAN,ALTAMIRO AMADEU SUSIN
3314,1999,A TOOL FOR ANALYSIS OF UNIVERSAL LOGIC GATES F...,A TOOL FOR ANALYSIS OF UNIVERSAL LOGIC GATES F...,,,NAN,FLAVIO RECH WAGNER
3314,1999,A TOOL FOR ANALYSIS OF UNIVERSAL LOGIC GATES F...,A TOOL FOR ANALYSIS OF UNIVERSAL LOGIC GATES F...,,,NAN,MARIANA LUDERITZ KOLBERG
3314,1999,A TOOL FOR ANALYSIS OF UNIVERSAL LOGIC GATES F...,A TOOL FOR ANALYSIS OF UNIVERSAL LOGIC GATES F...,,,NAN,CARLOS EDUARDO PEREIRA
3314,1999,A TOOL FOR ANALYSIS OF UNIVERSAL LOGIC GATES F...,A TOOL FOR ANALYSIS OF UNIVERSAL LOGIC GATES F...,,,NAN,LUCIANA SALETE BURIOL
3314,1999,A TOOL FOR ANALYSIS OF UNIVERSAL LOGIC GATES F...,A TOOL FOR ANALYSIS OF UNIVERSAL LOGIC GATES F...,,,NAN,DANTE AUGUSTO COUTO BARONE
3314,1999,A TOOL FOR ANALYSIS OF UNIVERSAL LOGIC GATES F...,A TOOL FOR ANALYSIS OF UNIVERSAL LOGIC GATES F...,,,NAN,LISANDRO ZAMBENEDETTI GRANVILLE


In [9]:
## Comita as mudanças para tabelas do banco.
import mysql.connector

cnx = mysql.connector.connect(user='root', password='root', database='EBD10')
cursor = cnx.cursor()

researcher_insert = """INSERT INTO Researcher(researcher_name)
                        SELECT * FROM (SELECT %s) AS tmp
                        WHERE NOT EXISTS(
                           SELECT researcher_id from Researcher where researcher_name = %s
                        ) LIMIT 1;"""

field_insert = """INSERT INTO ResearchField (researchfield_name)
                    SELECT * FROM (SELECT %s) AS tmp
                    WHERE NOT EXISTS (
                        SELECT researchfield_name FROM ResearchField WHERE researchfield_name = %s
                    ) LIMIT 1;"""

program_insert = """INSERT INTO ResearchProgram (program_name)
                    SELECT * FROM (SELECT %s) AS tmp
                    WHERE NOT EXISTS (
                        SELECT program_name FROM ResearchProgram WHERE program_name = %s
                    ) LIMIT 1;"""

qualis_insert = """INSERT INTO Qualis(qualis)
                        SELECT * FROM (SELECT %s) AS tmp
                        WHERE NOT EXISTS(
                           SELECT qualis_id from Qualis where qualis = %s
                        ) LIMIT 1;"""

time_insert = """INSERT INTO Time(time_year)
                        SELECT * FROM (SELECT %s) AS tmp
                        WHERE NOT EXISTS(
                           SELECT time_id from Time where time_year = %s
                        ) LIMIT 1;"""

for idx, row in researcher_df.iterrows():
    data = (row['name'], row['name'])
    cursor.execute(researcher_insert, data)
    data = (row['research_area'], row['research_area'])
    cursor.execute(field_insert, data)
    data = (row['program'],row['program'])
    cursor.execute(program_insert, data)

for idx, row in production_df.iterrows():
    
    production_insert_start = """INSERT INTO Production (production_title, event_name"""
    production_insert_middle = """SELECT * FROM (SELECT %(title)s, %(event)s"""
    production_insert_end = """WHERE NOT EXISTS (
                            SELECT production_title FROM Production WHERE production_title = %(title)s and event_name =  %(event)s
                            ) LIMIT 1;"""
    data = {
        'title': row['TITULO-DO-TRABALHO'],
        'event': row['NOME-DO-EVENTO'],
        }
    
    if row['EVENT-ABREVIATION'] != '':
        production_insert_start = production_insert_start + ', event_abreviation'
        production_insert_middle = production_insert_middle + ', %(abrev)s'
        data['abrev'] = row['EVENT-ABREVIATION'] 
        
    if row['EVENT-ISSN'] != '':
        production_insert_start = production_insert_start + ', ISSN'
        production_insert_middle = production_insert_middle + ', %(issn)s'
        data['issn'] = row['EVENT-ISSN']
    
    production_insert_start = production_insert_start + ') '
    production_insert_middle = production_insert_middle + ') as tmp ' 
    production_insert = production_insert_start + production_insert_middle + production_insert_end
    
    cursor.execute(production_insert, data)
    
    data = (row['QUALIS'], row['QUALIS'])
    cursor.execute(qualis_insert, data)
    
    data = (row['ANO-DO-TRABALHO'], row['ANO-DO-TRABALHO'])
    cursor.execute(time_insert, data)
    


cnx.commit()
cursor.close()
cnx.close()

ProgrammingError: 1060 (42S21): Duplicate column name 'A TOOL FOR ANALYSIS OF UNIVERSAL LOGIC GATES FUNCTIONALITY'

In [ ]:
data

In [12]:
cnx.commit()
cursor.close()
cnx.close()

In [ ]:
type(data)